# Fetch books using google api

In [1]:
import requests
import json
import os

In [2]:
api = 'https://www.googleapis.com/books/v1/volumes'
base_folder = './topics'
os.listdir(base_folder)

['Computers & Technology',
 'Business & Investing',
 'Art, Music & Photography',
 'Science & Math',
 'Literature & Fiction',
 'Comics & Graphics Novels']

In [10]:
dir = 'Biographies & Memoirs'
folder = base_folder + '/' + dir
if dir not in os.listdir(base_folder):
    os.mkdir(folder) 
catg = ['Ethnic & Cultural', 'Europe', 'Historical', 'Leaders & Notable People', 'Military', 'Modern', 'Sports', 'United States', 'Women']

In [4]:
def fetch(subject, file, q=0, folder=folder, loop=3):
    arr = []
    api = 'https://www.googleapis.com/books/v1/volumes'
    for l in range(loop):
        if not q:
            url = api + '?q=subject:' + subject + '&langRestrict=en&startIndex=' + str(l * 40) + '&maxResults=40'
        else:
            url = api + '?q=' + subject + '&langRestrict=en&startIndex=' + str(l * 40) + '&maxResults=40'
        res = requests.get(url).json()
        if not res['totalItems']:
            break
        arr += res['items']
        if len(res['items']) < 40:
            break
    if len(arr):
        print(len(arr))
        with open(folder + '/' + file + '.json', 'w') as f:
            json.dump(arr, f)

In [15]:
file = catg[0]
print(file)
subject = file + '+biographies+memoirs'

Ethnic & Cultural


In [16]:
# url = 'https://www.googleapis.com/books/v1/volumes?q=subject:' +subject + '&langRestrict=en&maxResults=40'
url = 'https://www.googleapis.com/books/v1/volumes?q=' +subject + '&langRestrict=en&maxResults=40'
res = requests.get(url).json()
print(res['totalItems'])
if res['totalItems']:
    res = res['items']
    for i in range(0, min(40, len(res))):
        print('title:' + res[i]['volumeInfo']['title'])
        if 'categories' in res[i]['volumeInfo']:
            print('category: ' + str(res[i]['volumeInfo']['categories']))
        if 'description' in res[i]['volumeInfo']:
            print('description: ' + res[i]['volumeInfo']['description'][:200])
        print()

2789
title:EthniCity
category: ['Social Science']
description: Many urban areas in the developed world are ethnically diverse. They often compete for housing, employment, educational resources and political representation; may suffer residential segregation; but 

title:The International Spread of Ethnic Conflict
category: ['Political Science']
description: This work focuses on how, why and when ethnic conflicts either diffuse by precipitating similar conflicts elsewhere or escalate by bringing in outside parties and how such transnational ethnic conflic

title:Ethnic Conflict and International Security
category: ['Political Science']
description: 8. Ethnic conflict and refugees, by Kathleen Newland

title:Inter-ethnic and Religious Conflict Resolution in Nigeria
category: ['History']
description: Since 1982, Nigeria has experienced more than ten large scale ethnic or religious riots in its major cities. These violent clashes have wreaked economic, political, and social havoc; caused a

## Fetch Function

In [196]:
fetch(subject, file, q=0, loop=3)

120


## Fetching Manually

In [186]:
arr = []
url = api + '?q=subject:' + subject + '&langRestrict=en&startIndex=0&maxResults=40'
# url = api + '?q=' + subject + '&langRestrict=en&startIndex=0&maxResults=40'
res = requests.get(url).json()
arr += res['items']
len(arr)

17

In [189]:
# url = api + '?q=subject:' + subject + '&langRestrict=en&startIndex=40&maxResults=40'
url = api + '?q=' + subject + '&langRestrict=en&startIndex=40&maxResults=40'
res = requests.get(url).json()
arr += res['items']
len(arr)

97

In [116]:
# url = api + '?q=subject:' + subject + '&langRestrict=en&startIndex=80&maxResults=40'
url = api + '?q=' + subject + '&langRestrict=en&startIndex=80&maxResults=40'
res = requests.get(url).json()
arr += res['items']
len(arr)

120

In [190]:
with open(folder + '/' + file + '.json', 'w') as f:
    json.dump(arr, f)

# Json to CSV

In [7]:
import os
import json
import csv

base_folder = './topics'
book_list = []

label = 0

for folder in os.listdir(base_folder):
    folder_path = os.path.join(base_folder, folder)
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        with open(file_path) as f:
            books = json.load(f)
            for book in books:
                book = book['volumeInfo']
                title = book['title'] if 'title' in book else ""
                description = book['description'] if 'description' in book else ""
                if 'authors' in book:
                    authors = ''
                    for author in book['authors']:
                        authors += author + ", "
                    authors = authors.strip(', ')
                else:
                    authors = ''
                text = title + " " + authors + " " + description
                text = text.strip(' ')
                book_list.append({'label': label, 'category': folder + ' ' + file.split('.')[0], 'text': text })
        label += 1

with open('book_dataset.csv', 'w') as f:
    w = csv.DictWriter(f, book_list[0].keys())
    w.writeheader()
    w.writerows(book_list)